<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [316]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup

In [317]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 

DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [318]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [319]:
# текст запроса

query_3_1 = f'''
select count(id)
from vacancies
'''

In [320]:
# результат запроса

question_3_1 = pd.read_sql_query(query_3_1, connection)
display(question_3_1)

C:\Users\Евгений\AppData\Local\Temp\ipykernel_14888\2404565600.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  question_3_1 = pd.read_sql_query(query_3_1, connection)


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [321]:
# текст запроса

query_3_2 = '''
select count(id)
from employers
'''

In [322]:
# результат запроса

question_3_2 = pd.read_sql_query(query_3_2, connection)
display(question_3_2)

C:\Users\Евгений\AppData\Local\Temp\ipykernel_14888\771402731.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  question_3_2 = pd.read_sql_query(query_3_2, connection)


,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [323]:
# текст запроса

query_3_3 = '''
select count(id)
from areas
'''

In [324]:
# результат запроса

question_3_3 = pd.read_sql_query(query_3_3, connection)
display(question_3_3)

C:\Users\Евгений\AppData\Local\Temp\ipykernel_14888\1378987126.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  question_3_3 = pd.read_sql_query(query_3_3, connection)


,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [325]:
# текст запроса

query_3_4 = '''
select count(id)
from industries
'''

In [326]:
# результат запроса

question_3_4 = pd.read_sql_query(query_3_4, connection)
display(question_3_4)

C:\Users\Евгений\AppData\Local\Temp\ipykernel_14888\2088063869.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  question_3_4 = pd.read_sql_query(query_3_4, connection)


,count
0,294


***

In [327]:
# выводы по предварительному анализу данных

Чуть более 49 тысяч вакансий на 23,5 тысяч работодателей. Если учесть, что некоторые работодатели ищут работников сразу на несколько мест, то вакансий получается слишком мало. Но в реальности такое часто говорит о кризисе - значит, в столбце с работодателями много бесполезных для нас выбросов, либо уже неактульных позиций. Остальное пока подозрений не вызывает.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [328]:
# текст запроса

query_4_1 = '''
select 
    ar.name as "Имя региона", 
    count(vac.id) as "Количество вакансий"
from areas ar
join vacancies vac on ar.id = vac.area_id
group by ar.id, ar.name
order by 2 desc
'''

In [329]:
# результат запроса

question_4_1 = pd.read_sql_query(query_4_1, connection)
display(question_4_1)

C:\Users\Евгений\AppData\Local\Temp\ipykernel_14888\3445491532.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  question_4_1 = pd.read_sql_query(query_4_1, connection)


,Имя региона,Количество вакансий
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Кизляр,1
765,Джизак,1
766,Эртиль,1
767,Арсеньев,1


#### Примечание: для определения пятёрки лидеров в тексте запроса нужно добавить 'limit 5' в конце.

2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [330]:
# текст запроса

query_4_2 = '''
select
    count(*) as "Количество вакансий с ЗП"
    
from vacancies
where (salary_from is not null or salary_to is not null)
    or (salary_from is not null and salary_to is not null)
'''

In [331]:
# результат запроса

question_4_2 = pd.read_sql_query(query_4_2, connection)
display(question_4_2)

C:\Users\Евгений\AppData\Local\Temp\ipykernel_14888\4028830811.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  question_4_2 = pd.read_sql_query(query_4_2, connection)


,Количество вакансий с ЗП
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [332]:
# текст запроса

query_4_3 = '''
select
    round(AVG(salary_from)) as "Среднее нижней границы вилки",
    round(AVG(salary_to)) as "Среднее верхней границы вилки"
    
from vacancies
'''

In [333]:
# результат запроса

question_4_3 = pd.read_sql_query(query_4_3, connection)
display(question_4_3)

C:\Users\Евгений\AppData\Local\Temp\ipykernel_14888\907881813.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  question_4_3 = pd.read_sql_query(query_4_3, connection)


,Среднее нижней границы вилки,Среднее верхней границы вилки
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [334]:
# текст запроса

query_4_4 = '''
select
    schedule as "Тип рабочего графика",
    employment as "Тип трудоустройства",
    count(id) as "Количество вакансий"
from vacancies
group by schedule, employment
order by 3 desc
'''

In [335]:
# результат запроса

question_4_4 = pd.read_sql_query(query_4_4, connection)
display(question_4_4)

C:\Users\Евгений\AppData\Local\Temp\ipykernel_14888\1282468800.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  question_4_4 = pd.read_sql_query(query_4_4, connection)


,Тип рабочего графика,Тип трудоустройства,Количество вакансий
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [336]:
# текст запроса

query_4_5 = '''
select
    experience as "Требуемый опыт работы",
    count(id) as "Количество вакансий"
from vacancies
group by experience
order by 2
'''

In [337]:
# результат запроса

question_4_5 = pd.read_sql_query(query_4_5, connection)
display(question_4_5)

C:\Users\Евгений\AppData\Local\Temp\ipykernel_14888\3958980667.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  question_4_5 = pd.read_sql_query(query_4_5, connection)


,Требуемый опыт работы,Количество вакансий
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [338]:
# выводы по детальному анализу вакансий

1) Как можно заметить, часть вакансий не из России, но из СНГ.
2) Половина от всех вакансий не имеют установленного уровня ЗП.
3) Средняя вилка заработных плат - примерно от 70 до 110 тысяч рублей, при этом не стоит забывать, что часть вакансий из СНГ, следовательно, переведены из национальный валюты в рубли.
4) Традиционно, рабочий график и тип трудостройства "полный день - полная занятость" лидирует. Здесь ничего необычного. Но если взять данные из 5 запроса, можно заметить, что стажировка не особо популярна - вакансий, не требующих опыта работы примерно в 10 раз больше. Значит, работодатели готовы брать людей без опыта и сразу платить им деньги, требуя от них выполнения полноценных задач и вовлечения в процесс.
5) Количество вакансий, требующих серьёзного опыта в сфере - менее 3%. При этом вакансий, требующих минимальный опыт работы - больше половины. Значит, работодателям нужны люди в достаточно короткий срок и они не готовы пожертвовать временем для поиска специалистов с большим опытом, но и в большистве своём не готовы брать людей совсем без опыта.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [339]:
# текст запроса

query_5_1 = '''
select
    emp.name as "Работодатель",
    count(vac.id) as "Количество вакансий"
from vacancies vac
join employers as emp on vac.employer_id = emp.id
group by emp.name
order by 2 desc
'''

In [340]:
# результат запроса

question_5_1 = pd.read_sql_query(query_5_1, connection)
display(question_5_1)

C:\Users\Евгений\AppData\Local\Temp\ipykernel_14888\3484141387.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  question_5_1 = pd.read_sql_query(query_5_1, connection)


,Работодатель,Количество вакансий
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331
...,...,...
14761,им. Тьюринга,1
14762,LLC Novalab Tech,1
14763,Компания Мир упаковки,1
14764,Лифинцев Олег Иванович,1


Примечание: я не стал ограничивать данные только 1 и 5 местами, чтобы не терять информативности для возможного использования этого запроса в будущем.

P.S. Ответ - Яндекс - Газпромнефть

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [341]:
# текст запроса

query_5_2 = '''
select
    ar.name as "Название региона",
    count(distinct vac.id) as "Количество вакансий",
    count(distinct emp.id) as "Количество работодателей"
    
from areas as ar 
left join vacancies as vac on vac.area_id = ar.id
left join employers as emp on ar.id = emp.area

where vac.id is null
group by ar.name
order by 2,3 desc
'''

In [342]:
# результат запроса

question_5_2 = pd.read_sql_query(query_5_2, connection)
display(question_5_2)

C:\Users\Евгений\AppData\Local\Temp\ipykernel_14888\1010056950.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  question_5_2 = pd.read_sql_query(query_5_2, connection)


,Название региона,Количество вакансий,Количество работодателей
0,Россия,0,410
1,Казахстан,0,207
2,Московская область,0,75
3,Краснодарский край,0,19
4,Беларусь,0,18
...,...,...,...
588,Кувандык,0,0
589,Куйбышев,0,0
590,Куйбышево (Ростовская область),0,0
591,Култаево,0,0


Ответ - "Россия".

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [343]:
# текст запроса

query_5_3 = '''
select
    emp.name as "Работодатель",
    count(distinct ar.id) as "Количество регионов"
    
from vacancies as vac
join employers as emp on vac.employer_id = emp.id
join areas as ar on vac.area_id = ar.id

group by emp.id
order by 2 desc
'''

In [344]:
# результат запроса

question_5_3 = pd.read_sql_query(query_5_3, connection)
display(question_5_3)

C:\Users\Евгений\AppData\Local\Temp\ipykernel_14888\34264136.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  question_5_3 = pd.read_sql_query(query_5_3, connection)


,Работодатель,Количество регионов
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14901,НПП Авиатрон,1
14902,Центр дистанционных торгов,1
14903,Городские Телекоммуникационные Системы,1
14904,"Введенский, Отель",1


Примечание. Не стал ограничивать данные 1-м местом, для возможности дальнейшего использования.

P.S. Ответ - 181.

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [345]:
# текст запроса

query_5_4 = '''
select
    count(*) as "Кол-во раб. без сфер деятельности"
    
from employers as emp
left join employers_industries as emp_ind on emp.id = emp_ind.employer_id
left join industries as ind on ind.id = emp_ind.industry_id

where ind.name is null
'''

In [346]:
# результат запроса

question_5_4 = pd.read_sql_query(query_5_4, connection)
display(question_5_4)

C:\Users\Евгений\AppData\Local\Temp\ipykernel_14888\3326048783.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  question_5_4 = pd.read_sql_query(query_5_4, connection)


,Кол-во раб. без сфер деятельности
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [347]:
# текст запроса

query_5_5 = '''
select emp.name as "Название компании"
    
from employers as emp
left join employers_industries as emp_ind on emp.id = emp_ind.employer_id
left join industries as ind on ind.id = emp_ind.industry_id

group by emp.id
having count(emp.id)=4
order by 1
'''

In [348]:
# результат запроса

question_5_5 = pd.read_sql_query(query_5_5, connection)
display(question_5_5)

C:\Users\Евгений\AppData\Local\Temp\ipykernel_14888\985117549.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  question_5_5 = pd.read_sql_query(query_5_5, connection)


,Название компании
0,101 Интернет
1,21vek.by
2,2ГИС
3,2К
4,4 пикселя +
...,...
1133,ЮРИОН
1134,ЮТИП Технологии
1135,ЯКласс
1136,ЯрНео


Примечание. Ответ - 2ГИС. Не стал ограничивать выборку для возможности дальнейшего использования.

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [349]:
# текст запроса

query_5_6 = '''
select count(*) as "Количестов разработчиков ПО"
    
from employers as emp
left join employers_industries as emp_ind on emp.id = emp_ind.employer_id
left join industries as ind on ind.id = emp_ind.industry_id

where ind.name = 'Разработка программного обеспечения'
'''

In [350]:
# результат запроса

question_5_6 = pd.read_sql_query(query_5_6, connection)
display(question_5_6)

C:\Users\Евгений\AppData\Local\Temp\ipykernel_14888\1450692104.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  question_5_6 = pd.read_sql_query(query_5_6, connection)


,Количестов разработчиков ПО
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [351]:
# код для получения списка городов-милионников

url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'

def get_million_cities(url):
    
    responce = requests.get(url)
    data = BeautifulSoup(responce.text, 'html.parser')
    wiki_table = str(data.find('table', class_='standard sortable'))
    wiki_table_values = wiki_table.split('title="')[3:]
    million_cities = []
    for item in wiki_table_values:
        million_cities.append(item.split('">')[0])
    return million_cities

million_cities = tuple(get_million_cities(url))

In [354]:
# текст запроса

query_5_7 = f'''
with tempT as (
select 
    ar.name as "Город",
    count(emp.id) as "Количество вакансий"
    
from employers as emp
join vacancies as vac on emp.id = vac.employer_id
join areas ar on vac.area_id = ar.id

where emp.name = 'Яндекс'
group by ar.name
having ar.name in {million_cities}
order by 2 desc
)

select 
    "Город",
    "Количество вакансий"
from tempT
union all
select 
    'Total',
    sum("Количество вакансий")
from tempT
    
'''

In [355]:
# результат запроса

question_5_7 = pd.read_sql_query(query_5_7, connection)
display(question_5_7)

C:\Users\Евгений\AppData\Local\Temp\ipykernel_14888\2332806867.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  question_5_7 = pd.read_sql_query(query_5_7, connection)


,Город,Количество вакансий
0,Москва,54.0
1,Санкт-Петербург,42.0
2,Екатеринбург,39.0
3,Нижний Новгород,36.0
4,Новосибирск,35.0
5,Воронеж,32.0
6,Краснодар,30.0
7,Самара,26.0
8,Уфа,26.0
9,Казань,25.0


***

In [ ]:
# выводы по анализу работодателей

1) Яндекс - абсолютный доминант по количеству вакансий. Причем у него куча вакансий в разных городах и регионах. Думаю, тут стоит задуматься о дополнительном фильтре или странице исключительно для него, так как последующие компании из топ-5 вместе взятые не могут с ним тягаться.
2) У нас имеется достаточно много работодателей, у которых нет активных вакансий. Нужно их немного детальнее исследовать: когда зарегистрировались и когда была последняя активность. От этого будет зависеть нужность их для будущей нейросети, там может быть очень много выбросов.
3) То же самое о работодателях, у которых не указана сфера деятельности. Иначе не для всех вакансий будет корректно работать подбор, если работодатель просто забыл указать этот пункт. Соответственно, точность нейросети пострадает.
4) Достаточно много компаний, у которых указано много сфер деятельности (4 и более).
5) Выяснилось, что примерно 1/4 работодателей - разработчики ПО. Один из них - вышеупомянутый Яндекс, плюс в топ-10 почти все имеют этот статус, если немного углубиться в эти компании.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [ ]:
# текст запроса

In [ ]:
# результат запроса

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [ ]:
# текст запроса

In [ ]:
# результат запроса

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
# текст запроса

In [ ]:
# результат запроса

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# текст запроса

In [ ]:
# результат запроса

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
# текст запроса

In [ ]:
# результат запроса

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [ ]:
# текст запроса

In [ ]:
# результат запроса

***

In [ ]:
# выводы по предметному анализу

# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования